In [1]:
from neo4j import GraphDatabase
import pandas as pd
from tabulate import tabulate
import os
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [2]:
directory = os.getcwd()
path = directory + '/Data/'

In [3]:
driver=GraphDatabase.driver("bolt://localhost:7687",auth=("neo4j","Prerna"))
session = driver.session(database="graphoptions")

In [4]:
result={"label":[],"count":[]}
with driver.session(database="graphoptions") as session:
    labels=[row["label"] for row in session.run("CALL db.labels()")]
    for label in labels:
        query=f"MATCH (:`{label}`) RETURN count(*) as count"
        count=session.run(query).single()["count"]
        result["label"].append(label)
        result["count"].append(count)
df=pd.DataFrame(data=result)
print(tabulate(df.sort_values("count"), headers='keys',tablefmt='psql',showindex=False))

+---------+---------+
| label   |   count |
|---------+---------|
| company |    6169 |
+---------+---------+


## Degree centrality: Counts outgoing relationships
Detects the number of direct connections a node has


In [5]:
pd.set_option('display.max_rows', 500)
fig=plt.figure()
query="""CALL gds.degree.stream({
nodeProjection: 'company',
relationshipProjection: {Holds: {type: 'Holds'}}})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS cik,gds.util.asNode(nodeId).company AS name, score as Degree_natural
ORDER BY Degree_natural DESC
"""

with driver.session(database="graphoptions") as session:
    Degree_natural=pd.DataFrame([dict(record) for record in session.run(query)])
    Degree_natural=Degree_natural[['cik','name','Degree_natural']]
Degree_natural

,cik,name,Degree_natural
0,1488542,SIMPLEX TRADING LLC,7532.0
1,1423053,CITADEL ADVISORS LLC,7104.0
2,1446194,SUSQUEHANNA INTERNATIONAL GRP LLP,7025.0
3,93254,GRP ONE TRADING LP,5834.0
4,1595888,JANE STREET GRP LLC,4256.0
...,...,...,...
6164,1486159,OASIS PETROLEUM INC.,0.0
6165,1486957,"BWX TECHNOLOGIES, INC.",0.0
6166,1487712,AIR LEASE CORP,0.0
6167,1492298,"SABRA HEALTH CARE REIT, INC.",0.0


In [6]:
query="""CALL gds.degree.stream({
nodeProjection: 'company',
relationshipProjection: {Holds: {type: 'Holds'}}, orientation: 'REVERSE'  })
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS cik,   score as Degree_reverse
ORDER BY Degree_reverse DESC
"""

with driver.session(database="graphoptions") as session:
    Degree_reverse=pd.DataFrame([dict(record) for record in session.run(query)])
    Degree_reverse=Degree_reverse[['cik','Degree_reverse']]
Degree_reverse

,cik,Degree_reverse
0,1100663,1229.0
1,884394,806.0
2,1064641,665.0
3,1424958,295.0
4,113736,290.0
...,...,...
6164,1679543,0.0
6165,1680843,0.0
6166,1680964,0.0
6167,1681614,0.0


In [7]:
query='''match ()-[r:Holds]->()
where r.value = toString(r.value)
set r.value = toInteger(r.value)
return count(r)'''
with driver.session(database="graphoptions") as session:
    session.run(query)

In [8]:
query="""CALL gds.degree.stream(({
  nodeProjection: 'company',
  relationshipProjection: {
    relType: {
      type: 'Holds',
      orientation: 'NATURAL',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value'
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik,  score as Degree_natural_wt
RETURN cik,  Degree_natural_wt
ORDER BY Degree_natural_wt DESC
"""

with driver.session(database="graphoptions") as session:
    Degree_natural_wt=pd.DataFrame([dict(record) for record in session.run(query)])
    Degree_natural_wt=Degree_natural_wt[['cik','Degree_natural_wt']]
Degree_natural_wt

,cik,Degree_natural_wt
0,1446194,2.598166e+12
1,1423053,3.943942e+11
2,1595888,2.502541e+11
3,93254,1.189069e+11
4,1452861,1.177423e+11
...,...,...
6164,1353613,0.000000e+00
6165,1362333,0.000000e+00
6166,1368286,0.000000e+00
6167,1368671,0.000000e+00


In [9]:
query="""CALL gds.degree.stream(({
  nodeProjection: 'company',
  relationshipProjection: {
    relType: {
      type: 'Holds',
      orientation: 'REVERSE',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value'
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik,   score as Degree_reverse_wt
RETURN cik,  Degree_reverse_wt
ORDER BY Degree_reverse_wt DESC
"""

with driver.session(database="graphoptions") as session:
    Degree_reverse_wt=pd.DataFrame([dict(record) for record in session.run(query)])
    Degree_reverse_wt=Degree_reverse_wt[['cik','Degree_reverse_wt']]
Degree_reverse_wt

,cik,Degree_reverse_wt
0,884394,2.396192e+12
1,1318605,3.125894e+11
2,1100663,1.997988e+11
3,1067839,1.937822e+11
4,1018724,1.644437e+11
...,...,...
6164,1762068,0.000000e+00
6165,1762718,0.000000e+00
6166,1764008,0.000000e+00
6167,1764386,0.000000e+00


## Betweenness(unweighted)
A way of detecting the amount of influence a node has over the flow of information in a graph


In [10]:
query='''CALL gds.betweenness.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'NATURAL',
      properties: {}
    }
  }
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik,  score as Betweeness
RETURN cik,Betweeness
ORDER BY Betweeness DESC
'''

with driver.session(database="graphoptions") as session:
    Betweeness=pd.DataFrame([dict(record) for record in session.run(query)])
    Betweeness=Betweeness[['cik','Betweeness']]
Betweeness

,cik,Betweeness
0,19617,45169.285067
1,831001,40537.068918
2,886982,40250.202280
3,161052,20017.089307
4,312069,15936.166246
...,...,...
6164,1547459,0.000000
6165,1549084,0.000000
6166,1549107,0.000000
6167,1549595,0.000000


In [11]:
query='''CALL gds.betweenness.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'REVERSE',
      properties: {}
    }
  }
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik,  score as Betweeness_reverse
RETURN cik, Betweeness_reverse
ORDER BY Betweeness_reverse DESC
'''

with driver.session(database="graphoptions") as session:
    Betweeness_reverse=pd.DataFrame([dict(record) for record in session.run(query)])
    Betweeness_reverse=Betweeness_reverse[['cik','Betweeness_reverse']]
Betweeness_reverse

,cik,Betweeness_reverse
0,19617,45169.285067
1,831001,40537.068918
2,886982,40250.202280
3,161052,20017.089307
4,312069,15936.166246
...,...,...
6164,1547459,0.000000
6165,1549084,0.000000
6166,1549107,0.000000
6167,1549595,0.000000


## Approx betweeness
calculates shortest paths between a subset of nodes, unlike Betweenness which considers all pairs of nodes

In [12]:
query='''CALL gds.betweenness.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'NATURAL',
      properties: {}
    }
  },
  samplingSize: 100
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik,  score 
RETURN cik,  score as Ap_Betweeness_natural
ORDER BY Ap_Betweeness_natural DESC
'''

with driver.session(database="graphoptions") as session:
    Ap_Betweeness_natural=pd.DataFrame([dict(record) for record in session.run(query)])
    Ap_Betweeness_natural=Ap_Betweeness_natural[['cik','Ap_Betweeness_natural']]
Ap_Betweeness_natural

,cik,Ap_Betweeness_natural
0,19617,1045.078097
1,886982,1035.792026
2,831001,625.980931
3,161052,529.172194
4,312069,379.310263
...,...,...
6164,100498,0.000000
6165,1005284,0.000000
6166,1005731,0.000000
6167,1005897,0.000000


In [13]:
query='''CALL gds.betweenness.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'REVERSE',
      properties: {}
    }
  },
  samplingSize: 100
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name,  score
RETURN cik,  score as Ap_Betweeness_reverse
ORDER BY Ap_Betweeness_reverse DESC
'''

with driver.session(database="graphoptions") as session:
    Ap_Betweeness_reverse=pd.DataFrame([dict(record) for record in session.run(query)])
    Ap_Betweeness_reverse=Ap_Betweeness_reverse[['cik','Ap_Betweeness_reverse']]
Ap_Betweeness_reverse

,cik,Ap_Betweeness_reverse
0,831001,2371.996574
1,19617,2185.466864
2,886982,1656.125669
3,1000275,1395.647890
4,161052,518.576981
...,...,...
6164,1708646,0.000000
6165,51434,0.000000
6166,894089,0.000000
6167,1061937,0.000000


## Pagerank 
Measures the transitive influence or connectivity of nodes

In [14]:
query='''CALL gds.pageRank.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'NATURAL',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value',
  dampingFactor: 0.85,
  maxIterations: 20
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name,  score
RETURN cik,  score as Page_Rank_wt_natural
ORDER BY Page_Rank_wt_natural DESC
'''

with driver.session(database="graphoptions") as session:
    Page_Rank_wt_natural=pd.DataFrame([dict(record) for record in session.run(query)])
    Page_Rank_wt_natural=Page_Rank_wt_natural[['cik','Page_Rank_wt_natural']]
Page_Rank_wt_natural

,cik,Page_Rank_wt_natural
0,884394,21.066648
1,1100663,9.553695
2,1067839,4.807629
3,1318605,3.829669
4,320193,3.672113
...,...,...
6164,1056491,0.150000
6165,1056527,0.150000
6166,1056581,0.150000
6167,1056823,0.150000


In [15]:
query='''CALL gds.pageRank.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'REVERSE',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value',
  dampingFactor: 0.85,
  maxIterations: 20
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name,  score
RETURN cik,  score as Page_Rank_wt_reverse
ORDER BY Page_Rank_wt_reverse DESC
'''

with driver.session(database="graphoptions") as session:
    Page_Rank_wt_reverse=pd.DataFrame([dict(record) for record in session.run(query)])
    Page_Rank_wt_reverse=Page_Rank_wt_reverse[['cik','Page_Rank_wt_reverse']]
Page_Rank_wt_reverse

,cik,Page_Rank_wt_reverse
0,1446194,126.630536
1,1423053,104.218136
2,1488542,61.113722
3,93254,50.403068
4,1318757,45.813122
...,...,...
6164,1818221,0.150000
6165,1818331,0.150000
6166,182016,0.150000
6167,1820191,0.150000


## HITS
Link analysis algorithm that rates nodes based on two scores, a hub score and an authority score.

In [16]:
query='''CALL gds.alpha.hits.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'REVERSE',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value',
  hitsIterations: 20
})) YIELD nodeId, values
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name, values.auth AS authScore, values.hub AS hubScore
RETURN cik, authScore as authScore_r, hubScore as hubScore_r
ORDER BY authScore DESC'''
with driver.session(database="graphoptions") as session:
    HITS_wt_reverse=pd.DataFrame([dict(record) for record in session.run(query)])
    HITS_wt_reverse=HITS_wt_reverse[['cik','authScore_r','hubScore_r']]
HITS_wt_reverse

,cik,authScore_r,hubScore_r
0,1446194,0.576841,0.000000
1,1423053,0.543485,0.000000
2,927337,0.302468,0.000000
3,1488542,0.180308,0.000000
4,1388391,0.178434,0.000000
...,...,...,...
6164,1818221,0.000000,0.000299
6165,1818331,0.000000,0.010128
6166,182016,0.000000,0.000281
6167,1820191,0.000000,0.000330


In [17]:
query='''CALL gds.alpha.hits.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'NATURAL',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value',
  hitsIterations: 20
})) YIELD nodeId, values
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name, values.auth AS authScore, values.hub AS hubScore
RETURN cik, authScore as authScore_n, hubScore as hubScore_n
ORDER BY authScore DESC'''
with driver.session(database="graphoptions") as session:
    HITS_wt_natural=pd.DataFrame([dict(record) for record in session.run(query)])
    HITS_wt_natural=HITS_wt_natural[['cik','authScore_n','hubScore_n']]
HITS_wt_natural

,cik,authScore_n,hubScore_n
0,1100663,0.572864,0.000000
1,1424958,0.274058,0.000000
2,117461,0.215102,0.000000
3,884394,0.192288,0.000000
4,1064641,0.153216,0.000000
...,...,...,...
6164,1363561,0.000000,0.000109
6165,1376192,0.000000,0.000436
6166,1380443,0.000000,0.004733
6167,1385702,0.000000,0.001419


## Closeness, it is unweighted
Detect nodes that are able to spread information very efficiently through a graph

In [18]:
query='''CALL gds.alpha.closeness.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'NATURAL',
      properties: {}
    }
  }
})) YIELD nodeId, centrality AS score
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name, score
RETURN cik,score as Closeness_natural
ORDER BY Closeness_natural DESC
'''
with driver.session(database="graphoptions") as session:
    Closeness_natural=pd.DataFrame([dict(record) for record in session.run(query)])
    Closeness_natural=Closeness_natural[['cik','Closeness_natural']]
Closeness_natural

,cik,Closeness_natural
0,1045742,1.000000
1,1819987,1.000000
2,1390844,1.000000
3,1209797,1.000000
4,1091003,1.000000
...,...,...
6164,184244,0.202369
6165,1512922,0.172215
6166,1783036,0.172215
6167,1853816,0.172215


In [19]:
query='''CALL gds.alpha.closeness.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'REVERSE',
      properties: {}
    }
  }
})) YIELD nodeId, centrality AS score
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name, score
RETURN cik, score as Closeness_reverse
ORDER BY Closeness_reverse DESC
'''
with driver.session(database="graphoptions") as session:
    Closeness_reverse=pd.DataFrame([dict(record) for record in session.run(query)])
    Closeness_reverse=Closeness_reverse[['cik','Closeness_reverse']]
Closeness_reverse

,cik,Closeness_reverse
0,1045742,1.000000
1,1819987,1.000000
2,1390844,1.000000
3,1209797,1.000000
4,1091003,1.000000
...,...,...
6164,184244,0.202369
6165,1512922,0.172215
6166,1783036,0.172215
6167,1853816,0.172215


## Degree centrality for call options using projected graph

In [20]:
query='''call gds.graph.drop('put')'''
with driver.session(database="graphoptions") as session:
    session.run(query)

In [21]:
query='''call gds.graph.drop('call')'''
with driver.session(database="graphoptions") as session:
    session.run(query)

In [22]:
query='''CALL gds.graph.create.cypher(
  'put',
  ' MATCH (n) RETURN toInteger(properties(n).id) AS id, labels(n) AS labels',
  ' MATCH (n)-[r:Holds]->(m)
    WHERE  r.putCall="Put"
    RETURN toInteger(properties(n).id) as source, toInteger(properties(m).id) as target,type(r) AS type, r.value as putvalue
'
)
YIELD
  graphName AS graph, nodeCount AS nodes, relationshipCount AS rels'''
with driver.session(database="graphoptions") as session:
    session.run(query)


In [23]:
query='''CALL gds.graph.create.cypher(
  'call',
  ' MATCH (n) RETURN toInteger(properties(n).id) AS id, labels(n) AS labels',
  ' MATCH (n)-[r:Holds]->(m)
    WHERE  r.putCall="Call"
    RETURN toInteger(properties(n).id) as source, toInteger(properties(m).id) as target,type(r) AS type, r.value as callvalue
'
)
YIELD
  graphName AS graph, nodeCount AS nodes, relationshipCount AS rels'''
with driver.session(database="graphoptions") as session:
    session.run(query)

In [24]:
query="""CALL gds.degree.stream('call',(
{orientation: 'NATURAL'
,  relationshipWeightProperty: 'callvalue'
})) YIELD nodeId, score
WITH nodeId AS cik,  score as Degree_natural_wt_call
RETURN cik,  Degree_natural_wt_call
ORDER BY Degree_natural_wt_call DESC
"""

with driver.session(database="graphoptions") as session:
    Degree_natural_wt_call=pd.DataFrame([dict(record) for record in session.run(query)])
    Degree_natural_wt_call=Degree_natural_wt_call[['cik','Degree_natural_wt_call']]
Degree_natural_wt_call['cik']=Degree_natural_wt_call['cik'].apply(str)
Degree_natural_wt_call

,cik,Degree_natural_wt_call
0,1446194,2.293709e+12
1,1423053,1.859281e+11
2,1595888,1.263788e+11
3,93254,6.330993e+10
4,1452861,4.890373e+10
...,...,...
6164,1910419,0.000000e+00
6165,1910488,0.000000e+00
6166,318989,0.000000e+00
6167,49205,0.000000e+00


In [25]:
query="""CALL gds.degree.stream('call',(
{orientation: 'REVERSE'
,  relationshipWeightProperty: 'callvalue'
})) YIELD nodeId, score
WITH nodeId AS cik,  score as Degree_reverse_wt_call
RETURN cik,  Degree_reverse_wt_call
ORDER BY Degree_reverse_wt_call DESC
"""

with driver.session(database="graphoptions") as session:
    Degree_reverse_wt_call=pd.DataFrame([dict(record) for record in session.run(query)])
    Degree_reverse_wt_call=Degree_reverse_wt_call[['cik','Degree_reverse_wt_call']]
Degree_reverse_wt_call['cik']=Degree_reverse_wt_call['cik'].apply(str)
Degree_reverse_wt_call

,cik,Degree_reverse_wt_call
0,884394,2.126357e+12
1,1318605,1.465623e+11
2,1018724,7.651333e+10
3,1100663,6.382866e+10
4,1067839,5.793368e+10
...,...,...
6164,350894,0.000000e+00
6165,711404,0.000000e+00
6166,720005,0.000000e+00
6167,730263,0.000000e+00


## Degree centrality for put options using projected graph

In [26]:
query="""CALL gds.degree.stream('put',(
{orientation: 'NATURAL'
,  relationshipWeightProperty: 'putvalue'
})) YIELD nodeId, score
WITH nodeId AS cik,  score as Degree_natural_wt_put
RETURN cik,  Degree_natural_wt_put
ORDER BY Degree_natural_wt_put DESC
"""

with driver.session(database="graphoptions") as session:
    Degree_natural_wt_put=pd.DataFrame([dict(record) for record in session.run(query)])
    Degree_natural_wt_put=Degree_natural_wt_put[['cik','Degree_natural_wt_put']]
Degree_natural_wt_put['cik']=Degree_natural_wt_put['cik'].apply(str)
Degree_natural_wt_put

,cik,Degree_natural_wt_put
0,1446194,3.044569e+11
1,1423053,2.084661e+11
2,1595888,1.238753e+11
3,1273087,8.127663e+10
4,70858,7.398720e+10
...,...,...
6164,1353311,0.000000e+00
6165,1353312,0.000000e+00
6166,1356643,0.000000e+00
6167,1363561,0.000000e+00


In [27]:
query="""CALL gds.degree.stream('put',(
{orientation: 'REVERSE'
,  relationshipWeightProperty: 'putvalue'
})) YIELD nodeId, score
WITH nodeId AS cik,  score as Degree_reverse_wt_put
RETURN cik,  Degree_reverse_wt_put
ORDER BY Degree_reverse_wt_put DESC
"""

with driver.session(database="graphoptions") as session:
    Degree_reverse_wt_put=pd.DataFrame([dict(record) for record in session.run(query)])
    Degree_reverse_wt_put=Degree_reverse_wt_put[['cik','Degree_reverse_wt_put']]
Degree_reverse_wt_put['cik']=Degree_reverse_wt_put['cik'].apply(str)
Degree_reverse_wt_put

,cik,Degree_reverse_wt_put
0,884394,2.698347e+11
1,1318605,1.660271e+11
2,1100663,1.359701e+11
3,1067839,1.358485e+11
4,1018724,8.793036e+10
...,...,...
6164,910988,0.000000e+00
6165,920354,0.000000e+00
6166,938583,0.000000e+00
6167,161272,0.000000e+00


## Joining all the tables

In [28]:
dfs = [Degree_natural, Degree_reverse, Degree_natural_wt, Degree_reverse_wt, Betweeness, Betweeness_reverse, Ap_Betweeness_natural, Ap_Betweeness_reverse, Page_Rank_wt_natural, Page_Rank_wt_reverse, HITS_wt_reverse, HITS_wt_natural, Closeness_natural, Closeness_reverse,Degree_natural_wt_call,Degree_reverse_wt_call,Degree_natural_wt_put,Degree_reverse_wt_put]
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on='cik'), dfs)
df_final.to_csv(path+ 'df_final_graphoptions.csv', index=False)

In [29]:
df_final.head()

,cik,name,Degree_natural,Degree_reverse,Degree_natural_wt,Degree_reverse_wt,Betweeness,Betweeness_reverse,Ap_Betweeness_natural,Ap_Betweeness_reverse,...,authScore_r,hubScore_r,authScore_n,hubScore_n,Closeness_natural,Closeness_reverse,Degree_natural_wt_call,Degree_reverse_wt_call,Degree_natural_wt_put,Degree_reverse_wt_put
0,1488542,SIMPLEX TRADING LLC,7532.0,0.0,9.564343e+10,0.0,0.0,0.0,0.0,0.0,...,0.180308,0.0,0.0,0.180308,0.645842,0.645842,4.347912e+10,0.0,5.216431e+10,0.0
1,1423053,CITADEL ADVISORS LLC,7104.0,0.0,3.943942e+11,0.0,0.0,0.0,0.0,0.0,...,0.543485,0.0,0.0,0.543485,0.595162,0.595162,1.859281e+11,0.0,2.084661e+11,0.0
2,1446194,SUSQUEHANNA INTERNATIONAL GRP LLP,7025.0,0.0,2.598166e+12,0.0,0.0,0.0,0.0,0.0,...,0.576841,0.0,0.0,0.576841,0.588331,0.588331,2.293709e+12,0.0,3.044569e+11,0.0
3,93254,GRP ONE TRADING LP,5834.0,0.0,1.189069e+11,0.0,0.0,0.0,0.0,0.0,...,0.159275,0.0,0.0,0.159275,0.550376,0.550376,6.330993e+10,0.0,5.559699e+10,0.0
4,1595888,JANE STREET GRP LLC,4256.0,0.0,2.502541e+11,0.0,0.0,0.0,0.0,0.0,...,0.137305,0.0,0.0,0.137305,0.479535,0.479535,1.263788e+11,0.0,1.238753e+11,0.0
